In [1]:
import pandas as pd
import numpy as np
import re
import os
import pickle
import patsy

from phenom.design import Formula
from phenom.phenotype import Phenotype

import matplotlib.pyplot as plt

# helper functions 

In [2]:
def plotFunctionInterval(x, f, **kwargs):

    temp = f
    plt.plot(x, (f).mean(0), **kwargs)
    plt.fill_between(x,
                     (f).mean(0) - 2 * (f).std(0),
                     (f).mean(0) + 2 * (f).std(0), alpha=.4)

In [47]:
def absFunctionVal(design, samp, xraw, func, var1='pH', var2='mMacid', 
                 val1=[7,6.5,6,5.5,5], val2=[0, 5, 10, 20], axline=False, fontsize=10):
    param = []
    for i, ph in enumerate(val1):
        for j, mma in enumerate(val2):
            # build index for functions
            ind = [0]
            if not ph == 7:
                ind.append(design.columns.tolist().index('pH=%.1lf'%ph))
            if not mma == 0:
                ind.append(design.columns.tolist().index('mMAcid=%d'%mma))
            if not ph == 7 and not mma == 0:
                ind.append(design.columns.tolist().index('pH=%.1lf, mMAcid=%d'%(ph, mma)))

            # build mu samples
            mu = samp[:,ind,:].sum(1)
            
            r = [ph, mma]
            ev = func(mu)
            
            if ev.ndim == 1:
                r.extend([ev.mean(0), ev.std(0)])
            else:
                r.extend(ev.mean(0))
                r.extend(ev.std(0))
                
            param.append(r)
            
    return np.array(param)

In [49]:
def relFunctionVal(design, samp, xraw, func, var1='pH', var2='mMacid', 
                 val1=[7,6.5,6,5.5,5], val2=[0, 5, 10, 20], axline=False, fontsize=10):
    param = []

    for i, ph in enumerate([6.5,6,5.5,5]):
        for j, mma in enumerate([5, 10, 20]):

            ind = design.columns.tolist().index('pH=%.1lf, mMAcid=%d'%(ph, mma))
            mu = samp[:,ind,:]
            
            r = [ph, mma]
            ev = func(mu)
            
            if ev.ndim == 1:
                r.extend([ev.mean(0), ev.std(0)])
            else:
                r.extend(ev.mean(0))
                r.extend(ev.std(0))
                
            param.append(r)
            
    return np.array(param)

In [12]:
def plotAbsolute(design, samp, xraw, var1='pH', var2='mMacid', 
                 val1=[7,6.5,6,5.5,5], val2=[0, 5, 10, 20], ylabel='log(OD)', axline=False, fontsize=10):
    
    ylim = (np.inf, -np.inf)
    axes = []
    
    for i, ph in enumerate(val1):
        for j, mma in enumerate(val2):
            ind = [0]
            if not ph == 7:
                ind.append(design.columns.tolist().index('pH=%.1lf'%ph))
            if not mma == 0:
                ind.append(design.columns.tolist().index('mMAcid=%d'%mma))
            if not ph == 7 and not mma == 0:
                ind.append(design.columns.tolist().index('pH=%.1lf, mMAcid=%d'%(ph, mma)))

            # build mu samples
            mu = samp[:,ind,:].sum(1)
            
            ax = plt.subplot2grid((len(val2),len(val1)), (j, i))
            axes.append(ax)
            
            plotFunctionInterval(xraw, mu)
            title = ''
            if i == 0:
                plt.ylabel(ylabel, fontsize=fontsize)
                title += '%s = %s '%(var2, mma)
            if j == len(val2)-1:
                plt.xlabel('time (h)', fontsize=fontsize)
            if j == 0:
                title +='%s = %s'%(var1, ph)
            plt.title(title, fontsize=fontsize)
            if axline:
                plt.axhline(color='k')
            
            yl = ax.get_ylim()
            ylim = (min(ylim[0], yl[0]), max(ylim[1], yl[1]))
            
    for ax in axes:
        ax.set_ylim(ylim)

In [29]:
def plotRelative(design, samp, xraw, var1='pH', var2='mMacid', 
                 val1=[7,6.5,6,5.5,5], val2=[0, 5, 10, 20], ylabel='log(OD)', fontsize=12):
    
    ylim = (np.inf, -np.inf)
    
    for i, ph in enumerate([6.5,6,5.5,5]):
        for j, mma in enumerate([5, 10, 20]):

            ind = design.columns.tolist().index('pH=%.1lf, mMAcid=%d'%(ph, mma))
            mu = samp[:,ind,:]
            
            ax = plt.subplot2grid((3, 4), (j, i))
            axes.append(ax)
            plotFunctionInterval(xraw, mu)
            plt.axhline(color='k')
            
            title = ''
            if i == 0:
                plt.ylabel(ylabel, fontsize=fontsize)
                title += '%s = %s '%(var2, mma)
            if j == len(val2)-1:
                plt.xlabel('time (h)', fontsize=fontsize)
            if j == 0:
                title +='%s = %s'%(var1, ph)
            plt.title(title, fontsize=fontsize)

            yl = ax.get_ylim()
            ylim = (min(ylim[0], yl[0]), max(ylim[1], yl[1]))
        
    for ax in axes:
        ax.set_ylim(ylim)

# load posteriors 

In [6]:
posterior = {}

for strain, acid in [
        ('PA1054', 'butyric'), ('PA1054', 'citric'), 
        ('PA1054', 'potassium-sorbate'), ('PA1054', 'sodium-benzoate'),
        ('PA01', 'malic'), ('PA01', 'lactic'), ('PA01', 'citric'), 
        ('PA01', 'benzoate'), ('PA1054', 'propionic'), ('PA1054', 'acetic'), 
        ('PA01', 'potassium-sorbate'), ('PA01', 'butyric'), ('PA01', 'propionic'),
        ('PA01', 'acetic'), ('PA01', 'lactic'), ('PA1054', 'lactic'), 
        ('PA1054', 'malic'), ('PA01', 'benzoate'), ('PA01', 'citric'), ('PA01', 'lactic'), ('PA01', 'malic')]:
    
    label = '%s-%s'%(strain, acid)
    if not label in os.listdir('figures'):
        os.makedirs(os.path.join('figures', label))
    
    data = pd.read_csv(os.path.join('data/processed/', label, 'data.csv'), index_col=0)
    xmin, xmax = data.index.min(), data.index.max()
    
    samp = pickle.load(open(os.path.join('samples/', label, 'samples.pkl')))    
    data = data.iloc[:samp['f'].shape[2], :]
    samp['f'] *= np.nanstd(data.values)
    samp['f'][:,0] += np.nanmean(data.values)
    samp['df'] *= np.nanstd(data.values) / (xmax-xmin)
    
    posterior[label] = samp

# figures 

In [54]:
for strain, acid in [
        ('PA1054', 'butyric'), ('PA1054', 'citric'), 
        ('PA1054', 'potassium-sorbate'), ('PA1054', 'sodium-benzoate'),
        ('PA01', 'malic'), ('PA01', 'lactic'), ('PA01', 'citric'), 
        ('PA01', 'benzoate'), ('PA1054', 'propionic'), ('PA1054', 'acetic'), 
        ('PA01', 'potassium-sorbate'), ('PA01', 'butyric'), ('PA01', 'propionic'),
        ('PA01', 'acetic'), ('PA01', 'lactic'), ('PA1054', 'lactic'), 
        ('PA1054', 'malic'), ('PA01', 'benzoate'), ('PA01', 'citric'), ('PA01', 'lactic'), ('PA01', 'malic')]:
    
    label = '%s-%s'%(strain, acid)
    if not label in os.listdir('figures'):
        os.makedirs(os.path.join('figures', label))
    
    data = pd.read_csv(os.path.join('data/processed/', label, 'data.csv'), index_col=0)
    
    design = pd.read_csv(os.path.join('samples-temp', label, 'design.csv'), index_col=0)
    samp = posterior[label]
    
    data = data.iloc[:samp['f'].shape[2], :]
    xraw = data.index
    
    # abs plot
    plt.figure(figsize=(8,6))
    plotAbsolute(design, samp['f'], data.index, fontsize=10)
    plt.tight_layout()
    plt.savefig('figures/%s/abs.pdf'%label,bbox_inches='tight')
    plt.close()
    
    plt.figure(figsize=(8,6))
    plotAbsolute(design, samp['df'], data.index, ylabel='d log(OD) / dt', axline=True, fontsize=10)
    plt.tight_layout()
    plt.savefig('figures/%s/abs-deriv.pdf'%label,bbox_inches='tight')
    plt.close()
    
    # relative plots
    plt.figure(figsize=(8,6))
    plotRelative(design, samp['f'], xraw, fontsize=10)
    plt.tight_layout()
    plt.savefig('figures/%s/rel.pdf'%label,bbox_inches='tight')
    plt.close()
    
    plt.figure(figsize=(8,6))
    plotRelative(design, samp['df'], xraw, ylabel='d log(OD) / dt', fontsize=10)
    plt.tight_layout()
    plt.savefig('figures/%s/rel-deriv.pdf'%label,bbox_inches='tight')
    plt.close()

# parameters 

In [55]:
for strain, acid in [
        ('PA1054', 'butyric'), ('PA1054', 'citric'), 
        ('PA1054', 'potassium-sorbate'), ('PA1054', 'sodium-benzoate'),
        ('PA01', 'malic'), ('PA01', 'lactic'), ('PA01', 'citric'), 
        ('PA01', 'benzoate'), ('PA1054', 'propionic'), ('PA1054', 'acetic'), 
        ('PA01', 'potassium-sorbate'), ('PA01', 'butyric'), ('PA01', 'propionic'),
        ('PA01', 'acetic'), ('PA01', 'lactic'), ('PA1054', 'lactic'), 
        ('PA1054', 'malic'), ('PA01', 'benzoate'), ('PA01', 'citric'), ('PA01', 'lactic'), ('PA01', 'malic')]:
    
    label = '%s-%s'%(strain, acid)
    if not label in os.listdir('figures'):
        os.makedirs(os.path.join('figures', label))
    
    data = pd.read_csv(os.path.join('data/processed/', label, 'data.csv'), index_col=0)
    
    design = pd.read_csv(os.path.join('samples-temp', label, 'design.csv'), index_col=0)
    samp = posterior[label]
    
    data = data.iloc[:samp['f'].shape[2], :]
    xraw = data.index

    cc = absFunctionVal(design, samp['f'], xraw, lambda x: x.max(1))
    cc = pd.DataFrame(cc, columns=['ph', 'mMacid', 'mean', 'std'])
    cc.to_csv('figures/%s/carrycap.csv'%label)
    
    mm = absFunctionVal(design, samp['df'], xraw, lambda x: x.max(1))
    mm = pd.DataFrame(mm, columns=['ph', 'mMacid', 'mean', 'std']) 
    mm.to_csv('figures/%s/mumax.csv'%label)
    
    m = absFunctionVal(design, samp['f'], xraw, lambda x: x)
    m = pd.DataFrame(m, columns=['ph', 'mMacid'] + ['E[f(%.2lf)]'%xx for xx in xraw] +  ['std[f(%.2lf)]'%xx for xx in xraw]) 
    m.to_csv('figures/%s/function.csv'%label)
    
    m = absFunctionVal(design, samp['df'], xraw, lambda x: x)
    m = pd.DataFrame(m, columns=['ph', 'mMacid'] + ['E[f(%.2lf)]'%xx for xx in xraw] +  ['std[f(%.2lf)]'%xx for xx in xraw]) 
    m.to_csv('figures/%s/deriv.csv'%label)
    
    m = relFunctionVal(design, samp['f'], xraw, lambda x: x)
    m = pd.DataFrame(m, columns=['ph', 'mMacid'] + ['E[f(%.2lf)]'%xx for xx in xraw] +  ['std[f(%.2lf)]'%xx for xx in xraw]) 
    m.to_csv('figures/%s/function-rel.csv'%label)
    
    m = relFunctionVal(design, samp['df'], xraw, lambda x: x)
    m = pd.DataFrame(m, columns=['ph', 'mMacid'] + ['E[f(%.2lf)]'%xx for xx in xraw] +  ['std[f(%.2lf)]'%xx for xx in xraw]) 
    m.to_csv('figures/%s/deriv-rel.csv'%label)